In [260]:
from selenium import webdriver
import requests
from time import sleep
import datetime
import json
import pandas as pd
import os
import random
import sys

In [264]:
def get_cookie(user_):
    
    url_ = 'https://swufeapp.iswufe.info/jinzhi/index.php'
    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')  # # 浏览器不提供可视化页面

    # 拿到cookie
    # cookie有效期半小时左右
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url_)
    driver.find_element_by_xpath('//*[@id="username"]').send_keys(user_['name'])
    driver.find_element_by_xpath('//*[@id="password"]').send_keys(user_['password'])
    driver.find_element_by_xpath('//*[@id="casLoginForm"]/p[4]/button').click()
    sleep(2)
    
    # 打卡cookie
    # cookie_ = driver.get_cookies()[0]
    
    # 报备cookie
    driver.get('https://qxj.iswufe.info/QJ/XSBBList')
    sleep(1)
    cookie_ = driver.get_cookies()
    cookie_0 = cookie_[0]['name'] + '=' + cookie_[0]['value']
    cookie_1 = cookie_[1]['name'] + '=' + cookie_[1]['value']
    
    driver.quit()
    # return cookie_['name']+'='+cookie_['value']
    return cookie_0 + ';' + cookie_1

def get_profile(cookie_):
    
    url_ = 'https://qxj.iswufe.info/QJ/GetStuBBList'
    
    # 拿到个人基本信息,第一次时运行
    headers = {
        'cookie': cookie_,
        'Host': 'qxj.iswufe.info',
        'Origin': 'https://qxj.iswufe.info',
        'Referer': 'https://qxj.iswufe.info/QJ/XSBBList',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
    }
    
    # 请求获取个人信息
    response = requests.post(url_, headers=headers)
    t = response.text
    
    profile = json.loads(json.loads(t))[0]
    xh = profile['XH']
    xm = profile['XM']
    lxfs = profile['LXFS']
    xy = profile['XY']
    fdyid = profile['COUN_EMPLID']
    
    profile_dict = {
        'xh': xh,
        'xm': xm,
        'xy': xy,
        'lxfs': lxfs,
        'fdyid': fdyid
    }
    
    return profile_dict

def fix_variable(x_0, x_1, x_2):
    m = {
        'qjyy': x_0,
        'mdd': x_1,
        'qjxc': x_2
    }
    return m

# 提交报备请求
def post_bb(cookie_, data):
    
    url_ = 'https://qxj.iswufe.info/QJ/InsertStuBB'
    headers = {
        'cookie': cookie_,
        'Host': 'qxj.iswufe.info',
        'Origin': 'https://qxj.iswufe.info',
        'Referer': 'https://qxj.iswufe.info/QJ/XSBBList',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
    }
    response = requests.post(url_, headers=headers, data=data)
    return response.status_code
    
def save_log(log_):
    with open('{}_{}.txt'.format(log_['学号'], log_['timestamp']), 'w', encoding='gbk') as file:
        file.write(json.dumps(log_, ensure_ascii=False))

In [265]:
# 日志
log = {
    '学号': '',
    '获取cookie': '',
    '获取个人信息': '',
    '读取个人信息': '',
    '报备理由组合': '',
    '报备结果': '',
    'timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H-%M'),
    'time_cost': ''
}
start_time = datetime.datetime.now()

# 初始化账号、密码
user = {'name':'21902520005', 'password':'Lzb.981208'}
user_type = '5'

# 日志——学号
log['学号'] = user['name']

# 获取临时cookie
try:
    cookie_ = get_cookie(user)
    log['获取cookie'] = '成功_' + datetime.datetime.now().strftime('%H:%M:%S')
except:
    log['获取cookie'] = '失败_' + datetime.datetime.now().strftime('%H:%M:%S')
    save_log(log)
    sys.exit(0)
    
## 检查是否第一次登陆
# 如果不是第一次登陆，跳过
# 如果是第一次登陆，获得个人基本信息并保存成csv
try:
    if '{}.xlsx'.format(user['name']) in os.listdir():
        log['获取个人信息'] = '成功_已存在_' + datetime.datetime.now().strftime('%H:%M:%S')
        pass
    else:
        profile_ = get_profile(cookie_)
        pd.DataFrame(profile_, index=[0]).to_excel('{}.xlsx'.format(user['name']), index=False, encoding='gbk')
        log['获取个人信息'] = '成功_新获取_' + datetime.datetime.now().strftime('%H:%M:%S')
except:
    log['获取个人信息'] = '失败_' + datetime.datetime.now().strftime('%H:%M:%S')
    save_log(log)
    sys.exit(0)
    
# 读取个人信息
try:
    df_profile = pd.read_excel('{}.xlsx'.format(user['name']), dtype=object)
    log['读取个人信息'] = '成功_' + datetime.datetime.now().strftime('%H:%M:%S')
except:
    log['获取个人信息'] = '失败_' + datetime.datetime.now().strftime('%H:%M:%S')
    save_log(log)
    sys.exit(0)

## 报备理由组合
## 可变选项
variable_dict = {
    'qjyy': ['购买生活物资', '拿快递', '吃饭', '理发', '找导师'],
    'mdd': ['南门', '北二门', '文化宫', '柳林'],
    'qjxc': ['步行', '地铁', '公交', '打车']
}

## 固定选项
# 进出时间
now_time = datetime.datetime.now() - datetime.timedelta(minutes=5)
back_time = (now_time + datetime.timedelta(hours=2)).strftime('%H:%M')
now_time = now_time.strftime('%H:%M')
fixed_dict = {
    'qjlx': '5',
    'szxq': '光华校区',
    'upTemp': '',
    'qjkssj': now_time,
    'qjjssj': back_time,
}
# 加上个人信息
profile_.update(fixed_dict)

# 随机获得可变组合
if user_type == '0':
    variable_d = fix_variable('购买生活物资', '南门', '步行')
elif user_type == '1':
    variable_d = fix_variable('拿快递', '北二门', '步行')
elif user_type == '2':
    variable_d = fix_variable('找导师', '柳林', '地铁')
else:
    variable_d = {}
    for x in variable_dict.keys():
        variable_d.update({x:random.sample(variable_dict[x], 1)[0]})

# 获得最终上传表单
profile_.update(variable_d)
log['报备理由组合'] = profile_

# 报备
try:
    bb_status = post_bb(cookie_, profile_)
    if bb_status == 200:
        log['报备结果'] = '成功_' + datetime.datetime.now().strftime('%H:%M:%S')
        log['time_cost'] = str((datetime.datetime.now() - start_time).seconds)
        save_log(log)
    else:
        log['报备结果'] = '失败_' + str(bb_status) + '_' + datetime.datetime.now().strftime('%H:%M:%S')
        save_log(log)
except:
    log['报备结果'] = '失败_报错' + datetime.datetime.now().strftime('%H:%M:%S')
    save_log(log)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/var/folders/s2/hx0rrywn56l8fy0kbl1npysw0000gn/T/ipykernel_5519/1083094264.py", line 38, in <module>
    profile_ = get_profile(cookie_)
  File "/var/folders/s2/hx0rrywn56l8fy0kbl1npysw0000gn/T/ipykernel_5519/3074564146.py", line 48, in get_profile
    profile = json.loads(json.loads(t))[0]
  File "/Users/xiaokaikai/opt/anaconda3/envs/cookie/lib/python3.7/json/__init__.py", line 341, in loads
    raise TypeError(f'the JSON object must be str, bytes or bytearray, '
TypeError: the JSON object must be str, bytes or bytearray, not dict

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/xiaokaikai/opt/anaconda3/envs/cookie/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/s2/hx0rrywn56l8fy0kbl1npysw0000gn/T/ipykernel_5519/1083094264.py", line 44, in <module>
    sys.exit(

TypeError: object of type 'NoneType' has no len()